In [2]:
import pandas as pd
import numpy as np
import re
from sklearn.metrics import accuracy_score, f1_score, precision_score
from collections import Counter

In [43]:
#xlm_ita_age_train = pd.DataFrame(np.load('/g100_work/IscrC_mental/data/user_classification/trained_models/age/XLM_probs_age_traind.npy')
#xlm_ita_gender_train = pd.DataFrame(np.load('/g100_work/IscrC_mental/data/user_classification/trained_models/gender/XLM_probs_gender_train.npy')

xlm_ita_gender = pd.DataFrame(np.load('/g100_work/IscrC_mental/data/user_classification/trained_models/gender/XLM_probs_gender_test.npy'))
xlm_ita_gender.columns = ['user_id', 'p_is_female', 'p_is_male']
xlm_ita_gender.user_id = xlm_ita_gender.user_id.astype(float)

xlm_ita_age = pd.DataFrame(np.load('/g100_work/IscrC_mental/data/user_classification/trained_models/age/XLM_probs_age_test.npy'))
xlm_ita_age.columns = ['user_id', 'pred_age_0_19_prob', 'pred_age_20_29_prob', 'pred_age_30_39_prob', 'pred_age_40_100_prob']
xlm_ita_age.user_id = xlm_ita_age.user_id.astype(float)

m3_it = pd.read_pickle('/g100_work/IscrC_mental/data/user_classification/m3_scores_bio_image.pkl')

#cv_it = pd.read_pickle('/g100_work/IscrC_mental/data/user_classification/trained_models/cv_models.pkl')
#cv_it.user_id = cv_it.user_id.astype(float)

#cv_de = pd.read_pickle('/g100_work/IscrC_mental/data/user_classification/trained_models/german_age_and_gender.pkl')

test_set = pd.read_pickle('/g100_work/IscrC_mental/data/user_classification/data_for_models_test.pkl')
test_set.user_id = test_set.user_id.astype(float)


# remove extra instances from datasets
xlm_ita_gender = test_set.merge(xlm_ita_gender, on='user_id', how='inner')
xlm_ita_age = test_set.merge(xlm_ita_age, on='user_id', how='inner')
m3_it = test_set.merge(m3_it, on='user_id', how='inner')
#cv_it = test_set.merge(cv_it, on='user_id', how='inner')
# add missing rows in m3
m3_it = test_set.merge(m3_it, on='user_id', how='outer')
m3_it.loc[m3_it['score_male'].isna(), 'score_male'] = 0.5
m3_it.loc[m3_it['score_female'].isna(), 'score_female'] = 0.5
m3_it.loc[m3_it['score_age_cls_0'].isna(), 'score_age_cls_0'] = 0.25
m3_it.loc[m3_it['score_age_cls_0'].isna(), 'score_age_cls_0'] = 0.25
m3_it.loc[m3_it['score_age_cls_1'].isna(), 'score_age_cls_1'] = 0.25
m3_it.loc[m3_it['score_age_cls_2'].isna(), 'score_age_cls_2'] = 0.25
m3_it.loc[m3_it['score_age_cls_3'].isna(), 'score_age_cls_3'] = 0.25

In [21]:
# Assuming 'test_set' and 'xlm_ita_gender' are your DataFrames
test_set_user_ids = set(test_set['user_id'].astype(float))
xlm_ita_gender_user_ids = set(xlm_ita_gender['user_id'].astype(float))

# Find user IDs unique to 'test_set'
unique_to_test_set = test_set_user_ids.difference(xlm_ita_gender_user_ids)

# Find user IDs unique to 'xlm_ita_gender'
unique_to_xlm_ita_gender = xlm_ita_gender_user_ids.difference(test_set_user_ids)

# Combine the unique user IDs from both datasets
all_unique_user_ids = unique_to_test_set.union(unique_to_xlm_ita_gender)

# Create DataFrames with the unique user IDs
unique_test_set_df = test_set[test_set['user_id'].isin(unique_to_test_set)]
unique_xlm_ita_gender_df = xlm_ita_gender[xlm_ita_gender['user_id'].isin(unique_to_xlm_ita_gender)]

# Print the DataFrames containing unique user IDs
print("Unique User IDs in 'test_set':")
print(len(unique_to_test_set))
print("Unique User IDs in 'xlm_ita_gender':")
print(len(unique_to_xlm_ita_gender))
print("All unique User IDs:", len(all_unique_user_ids))
print("All unique User IDs:", len(set(test_set['user_id'].astype(int))))
print("All unique User IDs:", len(set(xlm_ita_gender['user_id'].astype(int))))

NameError: name 'xlm_ita_gender' is not defined

In [31]:
def twitter_features(
    df,
    include_bio=True,
    include_tweets=True,
    label_name='is_male',
    ):
    # Read the gold labels
    if label_name == 'is_male':
        gold_labels = df[label_name].tolist()
        gold_labels = ['male' if label == True else 'female' for label in gold_labels]
    if label_name == 'age':
        gold_labels = df[label_name].astype(int).tolist()
    if label_name == 'age_interval':
        # define age classes
        age_intervals = [0, 20, 30, 40, 100]
        age_labels = [0, 1, 2, 3]
        # Discretize the 'age' column into four classes
        gold_labels = pd.cut(df['age'], bins=age_intervals, labels=age_labels, right=False).astype('int').tolist()

    return _, gold_labels

In [41]:
# create df with gold labels for italian
test_set = pd.read_pickle('/g100_work/IscrC_mental/data/user_classification/data_for_models_test.pkl')
_ , gold_labels_age = twitter_features(test_set, label_name='age_interval')
_ , gold_labels_gender = twitter_features(test_set, label_name='is_male')
df_gold_labels = pd.DataFrame({"user_id": test_set.user_id.tolist(), "gold_gender": gold_labels_gender, "gold_age": gold_labels_age})
df_gold_labels.to_pickle("/g100_work/IscrC_mental/data/user_classification/trained_models/it/gold_labels_test.pkl")

In [30]:
# create df with gold labels for german
test_set = pd.read_pickle('/g100_work/IscrC_mental/data/user_classification/german_data/data_for_models_german_data.pkl')
_ , gold_labels_age = twitter_features(test_set, label_name='age_interval')
_ , gold_labels_gender = twitter_features(test_set, label_name='is_male')
df_gold_labels = pd.DataFrame({"user_id": test_set.user_id.tolist(), "gold_gender": gold_labels_gender, "gold_age": gold_labels_age})
df_gold_labels.to_pickle("/g100_work/IscrC_mental/data/user_classification/trained_models/de/gold_labels_test.pkl")

In [31]:
# create df with gold labels for dutch
test_set = pd.read_pickle('/g100_work/IscrC_mental/data/user_classification/dutch_data/data_for_models_dutch_data.pkl')
_ , gold_labels_age = twitter_features(test_set, label_name='age_interval')
_ , gold_labels_gender = twitter_features(test_set, label_name='is_male')
df_gold_labels = pd.DataFrame({"user_id": test_set.user_id.tolist(), "gold_gender": gold_labels_gender, "gold_age": gold_labels_age})
df_gold_labels.to_pickle("/g100_work/IscrC_mental/data/user_classification/trained_models/nl/gold_labels_test.pkl")

# Performance of majority-class dummy

In [34]:
_ , gold_labels = twitter_features(test_set)
print(Counter(gold_labels).most_common())
dum = ~(test_set.is_male.astype('bool')*False)
print('Ac dummy:',accuracy_score(test_set.is_male, dum)*100)
print('F1 dummy:',f1_score(test_set.is_male, dum, average='macro')*100)

[('male', 302), ('female', 82)]
Ac dummy: 78.64583333333334
F1 dummy: 44.02332361516035


In [33]:
test_set

,user_id,masked_bio,age,is_male,long_text
3,3430018155,,54,1.0,Bestanden !!!!\nWaffensachkunde Akademie Niede...
4,269803152,wohnort: wetzlat\r\r\n in gießen\r\r\nhobbys: ...,15,1.0,falls ihr eine neue hompage braucht erkundigt ...
5,1372423700,in düsseldorf,51,1.0,nice man!!\nme too\nsoo sorry\nhey how are you...
6,1184400427988803584,"mein name ist ömer özgüç, türkei , in thrakien...",58,1.0,Ein Stürmer wird dafür bezahlt Tore zu schieße...
7,2951843386,"in laband -gleiwitz , os. mit 6 j. einmarsch ...",80,1.0,Kriegskinder müssen den Friedenskindern erzähl...
...,...,...,...,...,...
601,1178286769353175040,bin kein spießer locker drauf,62,1.0,Thank you for All what your doing for jumanity...
602,407722041,ich bin habe alle höhen und tiefen durchlebt ...,56,1.0,R.I.P NICKI Goodbye Champion\nDas ist Ihre Cha...
603,3140335966,ich heiße noah. ich spiele fußball bin hab eu...,12,1.0,styler bri\n#FragAuba wie geht es dir in Dortm...
604,899250898127507456,❤😍,21,0.0,#♡ #Hashtag10\n#langeweile #l #lw #und #looks ...


In [47]:
_ , gold_labels = twitter_features(test_set, label_name='age_interval')
print(Counter(gold_labels).most_common())
dum = (np.array(gold_labels)*0)+3
print('Ac dummy:',accuracy_score(gold_labels, dum)*100)
print('F1 dummy:',f1_score(gold_labels, dum, average='macro')*100)

# compute average absolute error
mean_age_group1 = test_set[test_set.age<20].age.mean()
mean_age_group2 = test_set[(20<=test_set.age) & (test_set.age<30)].age.mean()
mean_age_group3 = test_set[(30<=test_set.age) & (test_set.age<40)].age.mean()
mean_age_group4 = test_set[(40<=test_set.age) & (test_set.age<=100)].age.mean()
class_means=[mean_age_group1, mean_age_group2, mean_age_group3, mean_age_group4]

if len(class_means) > 0:
    total_error = 0
    for i in range(len(gold_labels)):
        error = abs(class_means[3] - class_means[gold_labels[i]])
        total_error += error
    mean_error = total_error / len(gold_labels)
    
print('AAE dummy:', mean_error)

[(3, 572), (2, 220), (1, 217), (0, 110)]
Ac dummy: 51.117068811438784
F1 dummy: 16.9130691898285
AAE dummy: 14.954395470481234


In [35]:
def aggregate_performace(p1, p2, gold_labels, class_means=[]):
    
    gold_labels = np.array(gold_labels)

    # compute predictions
    pred1 = p1.argmax(axis=1) 
    pred2 = p2.argmax(axis=1)
    
    print(Counter(pred1))
    print(Counter(pred2))
    
    # assign majority class as default label
    majority_class = Counter(gold_labels).most_common()[0][0]
    rows_with_same_p1=np.where(np.all(p1 == 1/len(np.unique(gold_labels)), axis=1))[0]
    rows_with_same_p2=np.where(np.all(p2 == 1/len(np.unique(gold_labels)), axis=1))[0]
    if rows_with_same_p1.size > 0:
        print(f'{rows_with_same_p1.size} system1 predictions substituted with majority_class')
        pred1[rows_with_same_p1] = majority_class
    if rows_with_same_p2.size > 0:
        print(f'{rows_with_same_p2.size} system2 predictions substituted with majority_class')
        pred2[rows_with_same_p2] = majority_class
    
    print('**********')

    # compute acc of models 1 and 2
    acc1 = accuracy_score(gold_labels, pred1)
    acc2 = accuracy_score(gold_labels, pred2)

    # compute f1 of models 1 and 2
    f11 = f1_score(gold_labels, pred1, average=None)
    f12 = f1_score(gold_labels, pred2, average=None)
    
    
    if len(class_means) > 0:
        total_error1 = 0
        total_error2 = 0
        for i in range(len(gold_labels)):
            error1 = abs(class_means[pred1[i]] - class_means[gold_labels[i]])
            error2 = abs(class_means[pred2[i]] - class_means[gold_labels[i]])
            total_error1 += error1
            total_error2 += error2
        mean_error1 = total_error1 / len(gold_labels)
        mean_error2 = total_error2 / len(gold_labels)

    print('System 1')
    print('Ac:', acc1*100)
    print('F1:', f11.mean()*100)
    if len(class_means) > 0:
        print('MAE:', mean_error1)
    print('----------')
    print('System 2')
    print('Ac:', acc2*100)
    print('F1:', f12.mean()*100)
    if len(class_means) > 0:
        print('MAE:', mean_error2)
    print('----------')

    # compute aggregated predictions
    p_agg_1 = p1 + p2
    pred_agg_1 = p_agg_1.argmax(axis=1)
    acc_agg_1 = accuracy_score(gold_labels, pred_agg_1)
    f1agg_1 = f1_score(gold_labels, pred_agg_1, average=None)
    if len(class_means) > 0:
        total_error = 0
        for i in range(len(gold_labels)):
            error = abs(class_means[pred_agg_1[i]] - class_means[gold_labels[i]])
            total_error += error
        mean_error = total_error / len(gold_labels)
    print('Avg prediction system ')
    print('Ac:', acc_agg_1*100)
    print('F1:', f1agg_1.mean()*100)
    if len(class_means) > 0:
        print('MAE:', mean_error)
    print('----------')
    
    p_agg_2 = f11.mean()*p1 + f12.mean()*p2
    pred_agg_2 = p_agg_2.argmax(axis=1)
    acc_agg_2 = accuracy_score(gold_labels, pred_agg_2)
    f1agg_2 = f1_score(gold_labels, pred_agg_2, average=None)
    if len(class_means) > 0:
        total_error = 0
        for i in range(len(gold_labels)):
            error = abs(class_means[pred_agg_2[i]] - class_means[gold_labels[i]])
            total_error += error
        mean_error = total_error / len(gold_labels)
    print('F1mean-weighted prediction system ')
    print('Ac:', acc_agg_2*100)
    print('F1:', f1agg_2.mean()*100)
    if len(class_means) > 0:
        print('MAE:', mean_error)
    print('----------')

    p_agg_3 = f11*p1 + f12*p2
    pred_agg_3 = p_agg_3.argmax(axis=1)
    acc_agg_3 = accuracy_score(gold_labels, pred_agg_3)
    f1agg_3 = f1_score(gold_labels, pred_agg_3, average=None)
    if len(class_means) > 0:
        total_error = 0
        for i in range(len(gold_labels)):
            error = abs(class_means[pred_agg_3[i]] - class_means[gold_labels[i]])
            total_error += error
        mean_error = total_error / len(gold_labels)
    print('F1-weighted prediction system ')
    print('Ac:', acc_agg_3*100)
    print('F1:', f1agg_3.mean()*100)
    if len(class_means) > 0:
        print('MAE:', mean_error)
    print('----------')
    
    return pred_agg_1, pred_agg_2, pred_agg_3

# IT gender XLM+CV

In [23]:
# Assuming xlm_ita_gender is a Pandas Series or NumPy array
p1 = np.column_stack((xlm_ita_gender['p_is_male'], 1 - xlm_ita_gender['p_is_male']))

cv_it['p_is_male'] = cv_it.apply(lambda x: x.pred_gender_prob if bool(x.pred_is_male_label) else (1 - x.pred_gender_prob), axis=1)
p2 = np.column_stack((cv_it['p_is_male'], 1 - cv_it['p_is_male']))

gold_labels = (~test_set.is_male).astype(int)

aggregate_performace(p1, p2, gold_labels)

Counter({0: 747, 1: 372})
Counter({0: 875, 1: 244})
327 system2 predictions substituted with majority_class
**********
System 1
Ac: 88.11438784629134
F1: 86.84177720486235
----------
System 2
Ac: 75.96067917783735
F1: 70.64982424664947
----------
Avg prediction system 
Ac: 88.82931188561216
F1: 87.40755411713337
----------
F1mean-weighted prediction system 
Ac: 90.88471849865952
F1: 89.88998724489797
----------
F1-weighted prediction system 
Ac: 87.93565683646113
F1: 86.02240079943374
----------


# IT age XLM+CV

In [24]:
# Assuming xlm_ita_gender is a Pandas Series or NumPy array
p1 = np.array(xlm_ita_age[['pred_age_0_19_prob', 'pred_age_20_29_prob', 'pred_age_30_39_prob', 'pred_age_40_100_prob']])
p2 = np.array(cv_it[['pred_age_0_19_prob', 'pred_age_20_29_prob', 'pred_age_30_39_prob', 'pred_age_40_100_prob']])

_ , gold_labels = twitter_features(test_set, label_name='age_interval')

aggregate_performace(p1, p2, np.array(gold_labels), class_means=[mean_age_group1, mean_age_group2, mean_age_group3, mean_age_group4])

Counter({3: 591, 1: 244, 2: 219, 0: 65})
Counter({0: 591, 1: 382, 2: 88, 3: 58})
327 system2 predictions substituted with majority_class
**********
System 1
Ac: 66.39857015192135
F1: 60.15930806816667
MAE: 6.734919643762468
----------
System 2
Ac: 35.210008936550494
F1: 27.721998260576232
MAE: 16.244698119488422
----------
Avg prediction system 
Ac: 53.88739946380697
F1: 45.40926898500874
MAE: 10.076850951809869
----------
F1mean-weighted prediction system 
Ac: 64.96872207327972
F1: 57.39832210362758
MAE: 7.1666584028557185
----------
F1-weighted prediction system 
Ac: 65.59428060768543
F1: 55.3015265433231
MAE: 7.193943641759918
----------


In [25]:
Counter(gold_labels)

Counter({3: 572, 2: 220, 1: 236, 0: 91})

# IT gender XLM+M3

In [49]:
# Assuming xlm_ita_gender is a Pandas Series or NumPy array
p1 = np.column_stack((xlm_ita_gender['p_is_male'], 1 - xlm_ita_gender['p_is_male']))
p2 = np.array(m3_it[['score_male','score_female']])

gold_labels = (~test_set.is_male).astype(int)

gender_pred_agg_avg, gender_pred_agg_f1mean, _ = aggregate_performace(p1, p2, gold_labels)

Counter({0: 747, 1: 372})
Counter({0: 877, 1: 242})
115 system2 predictions substituted with majority_class
**********
System 1
Ac: 88.11438784629134
F1: 86.84177720486235
----------
System 2
Ac: 83.10991957104558
F1: 79.34006527449577
----------
Avg prediction system 
Ac: 92.22520107238606
F1: 91.22374303262394
----------
F1mean-weighted prediction system 
Ac: 91.86773905272565
F1: 90.8202369651584
----------
F1-weighted prediction system 
Ac: 91.06344950848973
F1: 89.77663920946652
----------


# IT age XLM+M3

In [50]:
# Assuming xlm_ita_gender is a Pandas Series or NumPy array
p1 = np.array(xlm_ita_age[['pred_age_0_19_prob', 'pred_age_20_29_prob', 'pred_age_30_39_prob', 'pred_age_40_100_prob']])
p2 = np.array(m3_it[['score_age_cls_0','score_age_cls_1','score_age_cls_2','score_age_cls_3']])

_ , gold_labels = twitter_features(test_set, label_name='age_interval')

age_pred_agg_avg, age_pred_agg_f1mean, _ = aggregate_performace(
    p1,
    p2,
    gold_labels,
    class_means=[mean_age_group1, mean_age_group2, mean_age_group3, mean_age_group4]
)

Counter({3: 591, 1: 244, 2: 219, 0: 65})
Counter({3: 591, 0: 226, 2: 164, 1: 138})
115 system2 predictions substituted with majority_class
**********
System 1
Ac: 66.13047363717605
F1: 59.573188746418296
MAE: 6.808219836607761
----------
System 2
Ac: 53.529937444146555
F1: 40.47351044742563
MAE: 11.972043431214914
----------
Avg prediction system 
Ac: 66.48793565683646
F1: 58.230413984506754
MAE: 7.2471894705657265
----------
F1mean-weighted prediction system 
Ac: 67.29222520107238
F1: 59.726974003760546
MAE: 6.871732394274091
----------
F1-weighted prediction system 
Ac: 66.84539767649687
F1: 57.27261849506693
MAE: 7.660232641180462
----------


In [51]:
pd.DataFrame(
    {
        "user_id": test_set.user_id,
        "gender_avg_agg": gender_pred_agg_avg,
        "gender_f1_agg": gender_pred_agg_f1mean,
        "age_avg_agg": age_pred_agg_avg,
        "age_f1_agg": age_pred_agg_f1mean
    }
).to_pickle('/g100_work/IscrC_mental/data/user_classification/trained_models/it/xlm_m3_untrained_agg_test.pickle')

In [52]:
df = pd.read_pickle('/g100_work/IscrC_mental/data/user_classification/trained_models/it/xlm_m3_untrained_agg_test.pickle')

In [53]:
df

,user_id,gender_avg_agg,gender_f1_agg,age_avg_agg,age_f1_agg
0,7.071362e+06,0,0,3,3
1,9.420092e+06,1,1,1,1
2,1.174941e+07,1,1,3,3
3,1.408858e+07,1,1,3,3
4,1.428183e+07,0,0,2,2
...,...,...,...,...,...
1115,1.492599e+18,0,0,0,0
1116,1.500578e+18,1,1,2,2
1117,1.511727e+18,0,0,3,3
1118,1.578278e+18,1,1,1,1
